In [57]:
# HIDDEN

# This file takes 'cleaned' .txt files from raw ASP program output.

import re
from collections import defaultdict
from ipywidgets import Button, HBox, VBox, Layout, widgets, interact, interactive
import ipywidgets
from IPython.display import display

In [65]:
# HIDDEN

def process(path):
    
    options = defaultdict(lambda: ({}, [])) # choices, reactions tuple
    tag = re.compile('\d')
    current_tag = 'start'
    
    with open(path, 'r') as f:
        lines = f.readlines()
        
        for line in lines:
            line = line.rstrip()
            if tag.match(line) is not None:
                current_tag = line
            elif 'decision' in line or 'continue' in line:
                choice, sub_tag = line.split('|')
                options[current_tag][0][choice] = sub_tag
            else:
                options[current_tag][1].append(line)
                
    return options

In [142]:
def print_story(text):
    string = ''
    for item in text:
        string += str(item)
        string += '<br />'
    return string

In [66]:
heist_18 = process('results/CLEAN-01-heist-18.txt')

In [70]:
heist_18['0']

({'decision(verb(search,trans(object(painting))))': '00',
  'decision(verb(search,trans(character(guard))))': '01'},
 ['setting(museum)',
  'story(rising)',
  'has(character(guard),object(key))',
  'is(character(guard),place(museum))',
  'is(character(thief),place(museum))',
  'is(object(painting),place(museum))'])

In [160]:
reactions = widgets.HTML(print_story(heist_18['start'][1]))
display(reactions)

HTML(value='setting(museum)<br />story(start)<br />has(character(guard),object(key))<br />is(character(guard),…

In [163]:
def f(choice):
    return choice
choose = interactive(f, choice = heist_18['start'][0])
display(choose)

interactive(children=(Dropdown(description='choice', options={'continue': '0'}, value='0'), Output()), _dom_cl…

In [164]:
button = Button(description = 'continue', icon='check')
display(button)

def click(test):
    reactions.value = print_story(heist_18[choose.kwargs['choice']][1])
    choose.children[0].options = heist_18[choose.result][0]

button.on_click(click)

Button(description='continue', icon='check', style=ButtonStyle())